# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Version Info
- 모델 학습 방법 : ViT
- Augmentation + 하이퍼파라미터 + 가중치 + 스케줄러 + weight decay + TTA + 홀드아웃 + 오프라인 증강

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [1]:
# 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [2]:
# 구글 드라이브에 업로드된 대회 데이터를 압축 해제하고 로컬에 저장합니다.
!tar -xvf drive/MyDrive/datasets_fin.tar > /dev/null

In [63]:
# 필요한 라이브러리를 설치합니다.
!pip install timm

In [ ]:
!pip install augraphy
!pip install matplotlib

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [21]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [22]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [ ]:
# [✅ V6 - 교체됨] Mixup이 적용된 'train_one_epoch'

# (Cell 14에서 정의한 final_norm_pipe를 사용)
norm_transform = final_norm_pipe 

def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = [] # (Mixup 라벨은 float이므로, acc/f1 계산이 어려움)

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        # --- [✅ V6 Mixup 로직] ---
        # 1. 50% 확률로 Mixup 적용
        if random.random() < 0.5:
            # 2. Mixup 수행 (Cell 14에 정의된 함수)
            # (image는 0-255 텐서, targets는 1차원 라벨)
            image_mixed, targets_mixed = mixup(image, targets, alpha=1.0)
            
            # 3. [✅ V6] Normalize/ToTensorV2를 여기서 수동 적용
            # (Albumentations의 Compose는 텐서를 못 받으므로, 수동 적용)
            image_mixed_norm = norm_transform(image=image_mixed.cpu().numpy().transpose(0, 2, 3, 1).astype(np.uint8))['image']
            image_to_train = image_mixed_norm.to(device)
            targets_to_train = targets_mixed.to(device) # (Mixup된 float 라벨)
            
        else:
            # 4. Mixup 미적용 (원본 학습)
            image_norm = norm_transform(image=image.cpu().numpy().transpose(0, 2, 3, 1).astype(np.uint8))['image']
            image_to_train = image_norm.to(device)
            # 5. [✅] Loss 계산을 위해 라벨을 원-핫으로 변경
            targets_to_train = torch.nn.functional.one_hot(targets, num_classes=17).float().to(device)
        # -----------------------------

        model.zero_grad(set_to_none=True)
        
        preds = model(image_to_train)
        
        # [✅ V6] Mixup 라벨(float)과 예측(logit)의 Loss 계산
        # (CrossEntropyLoss는 float 라벨을 못 받으므로, 수동 계산)
        loss = -torch.mean(torch.sum(targets_to_train * torch.log_softmax(preds, dim=1), dim=1))
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        # (Mixup 중에는 F1/Acc 계산이 부정확하므로, Loss만 추적)
        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)

    ret = {
        "train_loss": train_loss,
        "train_acc": 0, # (F1/Acc 계산 생략)
        "train_f1": 0,  # (F1/Acc 계산 생략)
    }
    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [24]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'datasets_fin/'

# model config
model_name = 'convnext_base' # 'efficientnet_b1' #'resnet50' 'resnet34' # , ...

# training config - 이 부분을 수정하여 성능을 높일 수 있음 
img_size = 224
LR = 3e-5
EPOCHS = 100
BATCH_SIZE = 8
num_workers = 2 # 속도 향상을 위해 조정
N_SPLITS = 5  # 5-Fold 교차 검증
EARLY_STOPPING_PATIENCE = 10 # 5 에포크 동안 점수 향상 없으면 중단


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [25]:
import cv2
import pandas as pd
from tqdm import tqdm
import albumentations as A
import os
from sklearn.model_selection import train_test_split

# 원본 CSV 불러오기
all_df = pd.read_csv("datasets_fin/train.csv")

# [✅ 핵심] 1570개를 8:2 비율로 '학습용'과 '미니 리더보드용'으로 분리
# stratify=all_df['target'] : 17개 클래스 비율을 유지하며 섞어줌 (Macro F1 문제 해결!)
train_df, holdout_df = train_test_split(
    all_df, 
    test_size=0.2,    # 20%를 미니 리더보드(holdout)로 사용
    random_state=42,  # 항상 동일하게 분리
    stratify=all_df['target'] 
)

# 학습은 80% 데이터로만 진행 (1570 * 0.8 = 약 1256개)
df = train_df.reset_index(drop=True) 

print(f"총 데이터: {len(all_df)}, K-Fold 학습용: {len(df)}, 미니 리더보드용: {len(holdout_df)}")




총 데이터: 1570, K-Fold 학습용: 1256, 미니 리더보드용: 314


In [ ]:
import albumentations as A
from albumentations.core.transforms_interface import ImageOnlyTransform
from albumentations.pytorch import ToTensorV2
from augraphy import * # [✅ V6] Augraphy 임포트
import cv2
import random
import numpy as np

# [✅ V6] 1. Custom Crop 함수 정의
class QuarterDivide(ImageOnlyTransform):
    def __init__(self, always_apply=False, p=1):
        super().__init__(always_apply=always_apply, p=p)
    def apply(self, img, **params):
        height, width, _ = img.shape
        center_x, center_y = width // 2, height // 2
        results = [
            img[0:center_y, 0:center_x], img[0:center_y, center_x:width],
            img[center_y:height, 0:center_x], img[center_y:height, center_x:width]
        ]
        return results[random.randint(0, 3)]

class HalfDivide(ImageOnlyTransform):
    def __init__(self, always_apply=False, p=1):
        super().__init__(always_apply=always_apply, p=p)
    def apply(self, img, **params):
        height, width, _ = img.shape
        center_y = height // 2
        results = [img[0:center_y, :], img[center_y:height, :]]
        return results[random.randint(0, 1)]

# [✅ V6] 2. Augraphy (물리 노이즈) 파이프라인
def augraphy_transform():
    return AugraphyPipeline(
        paper_phase = [
            OneOf([
                NoiseTexturize(sigma_range=(5, 15), p=0.5),
                BrightnessTexturize(texturize_range=(0.8, 0.99), p=0.5),
            ], p=0.6),
        ],
        post_phase = [
            OneOf([
                LightingGradient(direction=90, transparency=0.5, p=0.25),
                ShadowCast(shadow_side="bottom", shadow_blur_kernel_range=(101, 301), p=0.25)
            ], p=0.25),
        ]
    )

# [✅ V6] 4. Mixup 함수 정의 (학습 루프에서 사용)
def mixup(image, targets, alpha=1.0):
    indices = torch.randperm(image.size(0))
    image_b = image[indices]
    targets_b = targets[indices]
    
    lam = np.random.beta(alpha, alpha)
    
    # Mixup 이미지
    image_mixed = lam * image + (1 - lam) * image_b
    
    # Mixup 라벨 (원-핫 인코딩 후 섞기)
    targets_one_hot = torch.nn.functional.one_hot(targets, num_classes=17).float()
    targets_b_one_hot = torch.nn.functional.one_hot(targets_b, num_classes=17).float()
    targets_mixed = lam * targets_one_hot + (1 - lam) * targets_b_one_hot
    
    return image_mixed, targets_mixed

print("✅ [V6] 차세대 증강 함수(Augraphy, Extreme Crop, Mixup) 정의 완료!")

In [27]:
# Validation 함수
def validate_one_epoch(loader, model, loss_fn, device):
    model.eval() # 모델을 평가 모드로 설정
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad(): # 기울기 계산 비활성화
        pbar = tqdm(loader, desc="Valid")
        for image, targets in pbar:
            image = image.to(device)
            targets = targets.to(device)
            
            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())
            pbar.set_description(f"Valid Loss: {loss.item():.4f}")

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro') # Macro F1

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }
    return ret

In [ ]:
from PIL import Image

# [✅ V6 - 수정됨] '차세대 증강' 조건부 ImageDataset
class ImageDataset(Dataset):
    
    # [✅ 사용자님 분류 기준] (이전에 사용한 V5 리스트)
    PAPER_TARGETS = {1, 3, 4, 6, 7, 11, 12, 14, 15, 0, 10, 13} # '종이' 리스트

    def __init__(self, df, path, 
                 mode='train', # 'train', 'val', 'test' 모드
                 img_size=224, 
                 target_col='target'):
        
        self.df = df 
        self.path = path
        self.target_col = target_col
        self.mode = mode

        # --- [V6] 파이프라인을 __init__에서 한 번만 생성! ---
        if mode == 'train':
            self.augraphy_pipe = augraphy_transform()
            self.strong_geometric_pipe = strong_paper_geometric_transform(img_size, img_size)
            self.mild_pipe = light_non_paper_augment
            self.final_norm_pipe = final_norm_pipe # (Cell 14에 정의됨)
        else:
            self.val_pipe = val_transform # (Cell 14에 정의됨)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name = self.df.iloc[idx]['ID']
        target = self.df.iloc[idx][self.target_col]
        
        img_path = os.path.join(self.path, name)
        try:
            image = np.array(Image.open(img_path).convert("RGB"))
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            return torch.randn(3, img_size, img_size), 0 
            
        # --- [✅ V6 핵심 로직] ---
        if self.mode == 'train':
            if target in self.PAPER_TARGETS:
                image = self.augraphy_pipe(image=image)['image']
                image = self.strong_geometric_pipe(image=image)['image']
            else:
                image = self.mild_pipe(image=image)['image']
            
            # [✅ 중요] Normalize는 Mixup을 위해 학습 루프로 이동!
            # img = self.final_norm_pipe(image=image)['image']
            # -> V6는 Normalize/ToTensorV2를 Dataset에서 하지 않고,
            #    Mixup 직전에 수동으로 합니다.
            
            # [✅ V6] Normalize/ToTensorV2 대신, PIL Image -> Tensor로만 변환
            # (Mixup은 0-255 범위의 텐서를 섞어야 함)
            img = torch.from_numpy(image.transpose((2, 0, 1))).float() # (H, W, C) -> (C, H, W)
                
        else: # 'val' 또는 'test' 모드
            img = self.val_pipe(image=image)['image'] # val_pipe는 Normalize/ToTensorV2 포함
            
        return img, target

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [ ]:
# [✅ V6 - 수정됨] ImageDataset 생성

# 1. 학습 데이터 = 1256개 원본 (df)
# (V6는 '온라인' 증강이므로, 6280개가 아닌 1256개 원본(df)을 사용!)
print(f"학습용 원본 데이터(df) 로드 중... (총 {len(df)}개)")
trn_dataset = ImageDataset(
    df,                       # [✅ V6] aug_df -> df (1256개)
    "datasets_fin/train/",    # [✅ V6] train_aug -> train (원본 폴더)
    mode='train',             # [✅ V6] 'train' 모드
    img_size=img_size,        # [✅ V6] img_size 전달
    target_col='target'
)

# 2. 검증 데이터 = 314개 깨끗한 홀드아웃 (holdout_df)
print(f"검증용 홀드아웃(holdout_df) 로드 중... (총 {len(holdout_df)}개)")
val_dataset = ImageDataset(
    holdout_df,               
    "datasets_fin/train/",    
    mode='val',               # [✅ V6] 'val' 모드
    img_size=img_size,        # [✅ V6] img_size 전달
    target_col='target'
)

# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)

# [✅ V6] Loss 함수 수정 (Mixup을 위해)
# (Mixup은 train_one_epoch에서 Loss를 수동 계산하므로,
# loss_fn은 '검증용(validation)'으로만 사용됩니다.)
loss_fn = nn.CrossEntropyLoss() # (가중치 계산 삭제 - 단순화)

# ... (학습 루프는 동일) ...
# (학습 루프의 Train F1/Acc는 0으로 출력되지만, 
#  Valid F1 (Holdout) 점수가 '진짜' 점수입니다.)

# Optimizer, Scheduler
optimizer = Adam(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

# Early Stopping을 위한 변수
best_val_f1 = 0.0
patience_counter = 0

print(f"--- '미니 리더보드' 학습 시작 ---")
print(f"학습 데이터: {len(trn_dataset)}개 (증강됨)")
print(f"검증 데이터: {len(val_dataset)}개 (깨끗한 원본)")

for epoch in range(EPOCHS):
    start_time = time.time()

    train_ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    
    # [핵심] 314개의 깨끗한 홀드아웃 셋으로 검증
    val_ret = validate_one_epoch(val_loader, model, loss_fn, device)
    
    train_ret['epoch'] = epoch
    elapsed = time.time() - start_time
    scheduler.step()
    
    print(f"Epoch {epoch+1}/{EPOCHS} [{elapsed:.0f}s] - "
          f"Train F1: {train_ret['train_f1']:.4f}, "
          f"Valid F1 (Holdout): {val_ret['val_f1']:.4f}, " # <- 이게 진짜 점수
          f"Valid Loss: {val_ret['val_loss']:.4f}")

    # Early Stopping 및 모델 저장 로직
    if val_ret['val_f1'] > best_val_f1:
        best_val_f1 = val_ret['val_f1']
        
        # [✅ 수정됨] 저장 경로에서 {fold} 변수 제거
        torch.save(model.state_dict(), f"model/best_conv_model_old.pth") 
        print(f"⭐️ Best F1 Score updated to {best_val_f1:.4f}. Model saved!")
        patience_counter = 0 # 초기화
    else:
        patience_counter += 1
    
    if patience_counter >= EARLY_STOPPING_PATIENCE:
        print(f"Early stopping at epoch {epoch+1} as F1 score did not improve for {EARLY_STOPPING_PATIENCE} epochs.")
        break

print(f"--- 학습 완료 ---")
print(f"최종 '미니 리더보드' F1 점수: {best_val_f1:.4f}")
        

--- '미니 리더보드' 학습 시작 ---
학습 데이터: 6280개 (증강됨)
검증 데이터: 314개 (깨끗한 원본)


Valid Loss: 0.2632: 100%|██████████| 40/40 [00:06<00:00,  6.41it/s]


Epoch 1/100 [90s] - Train F1: 0.7945, Valid F1 (Holdout): 0.8321, Valid Loss: 0.3092
⭐️ Best F1 Score updated to 0.8321. Model saved!


Valid Loss: 0.2177: 100%|██████████| 40/40 [00:01<00:00, 37.67it/s]


Epoch 2/100 [80s] - Train F1: 0.9127, Valid F1 (Holdout): 0.9241, Valid Loss: 0.2137
⭐️ Best F1 Score updated to 0.9241. Model saved!


...

Epoch 29/100 [80s] - Train F1: 0.9946, Valid F1 (Holdout): 0.9225, Valid Loss: 0.3383


Valid Loss: 0.0001: 100%|██████████| 40/40 [00:01<00:00, 38.21it/s]


Epoch 30/100 [80s] - Train F1: 0.9954, Valid F1 (Holdout): 0.9293, Valid Loss: 0.3203


Valid Loss: 0.0026: 100%|██████████| 40/40 [00:01<00:00, 38.58it/s]

Epoch 31/100 [80s] - Train F1: 0.9966, Valid F1 (Holdout): 0.9263, Valid Loss: 0.3374
Early stopping at epoch 31 as F1 score did not improve for 10 epochs.
--- 학습 완료 ---
최종 '미니 리더보드' F1 점수: 0.9411


In [19]:
aug_df = pd.read_csv("datasets/aug_train.csv")

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [32]:
import numpy as np
import pandas as pd
import torch
import timm
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

# --- TTA 적용 '미니 리더보드' 점수 계산 시작 ---
print("--- TTA 적용 '미니 리더보드' 점수 계산 시작 ---")

# (주의: Cell 54에서 생성된 'holdout_df' 변수가 살아있어야 합니다)
if 'holdout_df' not in globals():
    print("오류: 'holdout_df'가 없습니다. Load Data 섹터를 다시 실행해야 할 수 있습니다.")
else:
    print(f"홀드아웃 데이터(holdout_df) 로드 중... (총 {len(holdout_df)}개)")

    # 1. TTA용 데이터로더 생성
    # [✅ 핵심] 'transform=tst_transform'을 사용합니다!
    holdout_tta_dataset = ImageDataset(
        holdout_df,               
        "datasets_fin/train/",    # 깨끗한 원본 이미지가 있는 'train' 폴더
        transform=tst_transform,  # TTA가 포함된 'tst_transform' 사용
        target_col='target'
    )

    holdout_tta_loader = DataLoader(
        holdout_tta_dataset,
        batch_size=BATCH_SIZE * 2, # 추론은 배치 2배로
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    # 2. 저장된 베스트 모델 불러오기
    model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
    model_path = "model/best_conv_model.pth" # 학습에서 저장된 경로
    model.load_state_dict(torch.load(model_path))
    model.eval()
    print(f"모델 {model_path} 로드 완료.")

    # 3. TTA 추론 실행 (Inference 섹션 코드 재활용)
    tta_preds_list = []
    with torch.no_grad():
        for image, _ in tqdm(holdout_tta_loader, desc="TTA 추론 중"):
            image = image.to(device)
            
            # TTA 0: 원본 예측
            pred_orig = model(image)

            # TTA 1: 좌우 반전
            image_hflip = torch.flip(image, dims=[-1]) 
            pred_hflip = model(image_hflip)
            
            # TTA 2: 90도 회전
            image_rot90 = torch.rot90(image, k=1, dims=[-2, -1])
            pred_rot90 = model(image_rot90)

            # 3개 예측의 확률(softmax)을 평균냄
            avg_batch_pred = (pred_orig.softmax(dim=1) + 
                              pred_hflip.softmax(dim=1) + 
                              pred_rot90.softmax(dim=1)) / 3.0
            
            tta_preds_list.append(avg_batch_pred.cpu().numpy())
    
    # (314, 17) 크기의 배열로 합치기
    final_tta_preds_array = np.concatenate(tta_preds_list, axis=0)

    # 4. TTA 적용 점수 계산
    # 최종 예측 라벨 (가장 확률 높은 것)
    final_tta_labels = np.argmax(final_tta_preds_array, axis=1)
    
    # 실제 정답 라벨
    true_labels = holdout_df['target'].values
    
    # Macro F1 스코어 계산
    tta_f1_score = f1_score(true_labels, final_tta_labels, average='macro')
    tta_acc_score = accuracy_score(true_labels, final_tta_labels)

    print("\n--- TTA 적용 '미니 리더보드' 최종 점수 ---")
    print(f"✅ TTA 적용 Macro F1: {tta_f1_score:.4f}")
    print(f"   (TTA 적용 Accuracy: {tta_acc_score:.4f})")
    
    # 홀드아웃 예측 확률 저장
    np.save("npy/conv_holdout_preds.npy", final_tta_preds_array)

--- TTA 적용 '미니 리더보드' 점수 계산 시작 ---
홀드아웃 데이터(holdout_df) 로드 중... (총 314개)
모델 model/best_conv_model.pth 로드 완료.


TTA 추론 중: 100%|██████████| 20/20 [00:02<00:00,  7.46it/s]


--- TTA 적용 '미니 리더보드' 최종 점수 ---
✅ TTA 적용 Macro F1: 0.9792
   (TTA 적용 Accuracy: 0.9809)


In [34]:
tst_df = pd.read_csv("datasets_fin/sample_submission.csv")
tst_dataset = ImageDataset(
    tst_df,
    "datasets_fin/test/",
    transform=tst_transform  # Test 셋에는 증강이 없는 val_transform 사용
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE * 2,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# 2. 저장된 베스트 모델 불러오기
model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
model_path = "model/best_conv_model_old.pth" # 학습에서 저장된 경로
model.load_state_dict(torch.load(model_path))
model.eval()
print(f"모델 {model_path} 로드 완료.")

# 3. TTA 추론 실행 (Inference 섹션 코드 재활용)
tta_preds_list = []
with torch.no_grad():
    for image, _ in tqdm(tst_loader, desc="TTA 추론 중"):
        image = image.to(device)
        
        # TTA 0: 원본 예측
        pred_orig = model(image)

        # TTA 1: 좌우 반전
        image_hflip = torch.flip(image, dims=[-1]) 
        pred_hflip = model(image_hflip)
        
        # TTA 2: 90도 회전
        image_rot90 = torch.rot90(image, k=1, dims=[-2, -1])
        pred_rot90 = model(image_rot90)

        # 3개 예측의 확률(softmax)을 평균냄
        avg_batch_pred = (pred_orig.softmax(dim=1) + 
                            pred_hflip.softmax(dim=1) + 
                            pred_rot90.softmax(dim=1)) / 3.0
        
        tta_preds_list.append(avg_batch_pred.cpu().numpy())

# 4. 최종 예측 라벨 생성
final_tta_preds_array = np.concatenate(tta_preds_list, axis=0)

np.save("npy/conv_final_preds.npy", final_tta_preds_array) 
print("ConvNeXt 평균 확률 저장 완료!")

final_tta_labels = np.argmax(final_tta_preds_array, axis=1)

# 5. [✅ 수정됨] F1 스코어 계산 부분 ***완전 삭제***
# (우리는 '진짜 테스트 데이터'의 정답을 모르므로 계산 불가능)
print("추론 완료. 제출 파일 생성 중...")

# 6. [✅ 수정됨] 제출 파일 생성
pred_df = tst_df.copy()
# [중요!] '가짜 정답'(true_labels)이 아닌, '모델의 예측'(final_tta_labels)을 넣어야 함
pred_df['target'] = final_tta_labels

모델 model/best_conv_model_old.pth 로드 완료.


TTA 추론 중: 100%|██████████| 197/197 [00:25<00:00,  7.81it/s]

ConvNeXt 평균 확률 저장 완료!
추론 완료. 제출 파일 생성 중...


In [20]:
print(final_tta_labels)

[ 2 12  5 ...  8  0 12]


In [13]:
assert (tst_df['ID'] == pred_df['ID']).all()

In [ ]:
pred_df.to_csv("submission/pred13.csv", index=False)
print("Submission file created!")

Submission file created!
